# Logistic Regression

## Model assumptions

- Outcome variable is categorical
- Observations are independent of each other
- No severe multicollinearity among X variables
- No extreme outliers
- Linear relationship between each X variable and the logit of the outcome variable
- Sufficiently large sample size

In [1]:
# Imports
import sys

# Adds the utils directory to the PYTHONPATH
sys.path.append("../utils")  

# Imports everything from common_imports.py
from common_imports import *  

# Adds the project root to the PYTHONPATH
sys.path.append("..")  

# Load cleaned dataset
from config import cleaned_data_path
df = pd.read_csv(cleaned_data_path)

ModuleNotFoundError: No module named 'common_imports'

In [ ]:
# Copy dataframe
df_enc = df.copy()

# Encode `salary` column as ordinal numeric category
salary_cat = ['low', 'medium', 'high']
df_enc['salary'] = (df_enc['salary'].astype('category').cat.set_categories(salary_cat).cat.codes)

# Dummy encode the `department` column
df_enc = pd.get_dummies(df_enc, drop_first=True)

# Display the new dataframe
df_enc.head()

Heatmap to visualize how correlated variables are.

In [ ]:
sub = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_monthly_hours', 'tenure']
fig = plt.subplots(figsize=(8, 4))
sns.heatmap(df_enc[sub].corr(), annot=True, cmap="crest")
plt.title('Heatmap of the dataset')
plt.show()

**Outliers:**

Since logistic regression is quite sensitive to outliers, it would be a good idea at this stage to remove the outliers in the tenure column that were identified earlier.

In [ ]:
# Compute 25th percentile
percent_25 = df['tenure'].quantile(0.25)

# Compute 75th percentile
percent_75 = df['tenure'].quantile(0.75)

# Compute interquantile range
iqr = percent_75 - percent_25

# Define upper and lower limits for non-oulier values in column
upper_limit = percent_75 + 1.5 * iqr
lower_limit = percent_25 - 1.5 * iqr

In [ ]:
# Select rows without outliers in `tenure` and save resulting dataframe in a new variable
df_logreg = df_enc[(df_enc['tenure'] >= lower_limit) & (df_enc['tenure'] <= upper_limit)]

# Display first few rows of new dataframe
df_logreg.head()

**Logistic Regression Model**

In [ ]:
# Construct and fit a logistic model
log_clf = LogisticRegression(random_state=42, max_iter=500).fit(X_train, y_train)

# Make predictions using the model
y_pred = log_clf.predict(X_test)

**Confusion Matrix**

In [ ]:
# Confusion Matrix
log_cm = confusion_matrix(y_test, y_pred, labels=log_clf.classes_)

# Display for the confusion matrix
log_disp = ConfusionMatrixDisplay(confusion_matrix=log_cm, display_labels=log_clf.classes_)

# Plot confusion matrix
log_disp.plot(values_format='')

# Display plot
plt.show()

The upper-left quadrant displays the number of true negatives. The upper-right quadrant displays the number of false positives. The bottom-left quadrant displays the number of false negatives. The bottom-right quadrant displays the number of true positives.

True negatives: The number of people who did not leave that the model accurately predicted did not leave.

False positives: The number of people who did not leave the model inaccurately predicted as leaving.

False negatives: The number of people who left that the model inaccurately predicted did not leave

True positives: The number of people who left the model accurately predicted as leaving

A perfect model would yield all true negatives and true positives, and no false negatives or false positives.

**Imbalance Check**

Checking the value counts in the `left` column. Since this is a binary classification task, the class balance informs the way you interpret accuracy metrics.

In [ ]:
df_logreg['left'].value_counts(normalize=True)

The split between the two classes is 83% to 17%, indicating that the data is not perfectly balanced but not too imbalanced. If the data was more severely imbalanced, resample it to achieve a more balanced distribution. However, in this instance, you can use the data without altering the class balance and proceed with model evaluation.

**Metrics Report**

In [ ]:
# Create classification report for logistic regression model
target_names = ['Predicted would not leave', 'Predicted would leave']
report = classification_report(y_test, y_pred, target_names=target_names, output_dict=True)

# Convert to dataframe 
report_df = pd.DataFrame(report).transpose()

# Display results
report_df

We are going to tailor the dataframe so as to be the base for the metrics that will be used in the other models

In [ ]:
## Extracting relevant data from the report_df

# Extracting the last row
results = report_df.tail(1).reset_index(drop=True)

# Rename 'f1-score' column
results = results.rename(columns={'f1-score' : 'F1'})

# Adding a new column "model" at the beginning with sample values
results.insert(0, 'model', ['logistic regression'])

# Dropping the last column ("support") from the DataFrame
results.drop(columns=['support'], inplace=True)

# Adding 'accuracy' and 'auc' columns to the DataFrame
results['accuracy'] = report_df.loc['accuracy'][0]
results['AUC'] = roc_auc_score(y_test, y_pred)

# Display the `result` dataframe
results

The table above shows that the logistic regression model achieved a precision of 79%, recall of 82%, f1-score of 80%, and accuracy of 82% with and area under the curve (AUC) of 60%.

## Tree-based Model

This approach covers implementation of Decision Tree and Random Forest.

## Model results

**1. Logistic Regression**

On the test set, the logistic regression model achieved the following:
- precision : 80%, 
- recall : 83%, 
- f1-score : 80% (all weighted averages), 
- accuracy : 83%, 
- AUC (area under the curve) : 60%

**2. Tree-based Machine Learning**

For both models, the test results were higher than the cross-validation's.

**Decision Tree**

On the test set, the decision tree model achieved the following:
- precision : 96%, 
- recall : 92%, 
- f1-score : 94% (all weighted averages), 
- accuracy : 98%, 
- AUC (area under the curve) : 96%

**Random Forest**

On the test set, the random forest model achieved the following:
- precision : 98%, 
- recall : 92%, 
- f1-score : 95% (all weighted averages), 
- accuracy : 98%, 
- AUC (area under the curve) : 95%

While the decision tree model has a higher AUC compared to the random forest, it can still be concluded that the random forest model is superior overall.


## Recommendations

To ensure that employees at the company are well-rested and remain satisfied, stakeholders should consider implementing the following recommendations.
- Limit the number of projects that employees can work on. 
- Promote employees who have been with the company for at least four years or investigate why these tenured employees are dissatisfied. 
- Stakeholders could either reward employees for working longer hours or limit their hours. 
- Inform employees about the company's overtime pay policies and clarify expectations around workload and time off. Company-wide and within-team discussions can also be held to address work culture. 
- Evaluation scores should be reserved for employees who work less than 200+ hours per month, and a proportionate scale should be used to reward employees who contribute more effort.